# IRVIZ in Jupyter

Demonstration of the IRVIZ Dash app running within a Jupyter environment (notebook or lab).

## Reading Data

The IRVIZ viewer requires that data is passed to it.

Typically, the user will need to provide a way (function) to read their data.
This function *should* return a dask array for better performance.

In this case, with the provided `BP-area3a.h5` file, we can use `irviz.app.open_map_file` to read the data and return a dask array.

In [ ]:
from dask import array as da
import h5py as h5
import numpy as np
import sklearn.decomposition

from irviz.app import open_map_file, open_ir_file

# Open the data
data_file =  "INSERT FILE PATH HERE" 
# data_file =  "./data/BP-area3a.h5"
data, bounds = open_map_file(data_file)

## Creating the Viewer (App)

The Viewer creates the Dash app if necessary, and create all of the visualization components to that app.

We will need to create a Viewer by using the `irviz.viewer.notebook_viewer` function.
This function has many optional arguments that can be passed to it to define which components get created, how they look, etc.

### Documentation

The documentation for the `notebook_viewer` and its arguments can be found by running:

`help(notebook_viewer)`

In [ ]:
from irviz.viewer import notebook_viewer

help(notebook_viewer)

### Optional Data and Arguments for the Viewer

Below we will be preparing optional data and arguments for the viewer.

#### Decomposition and Component Labeling

In this example, we will use PCA.

In [ ]:
model = sklearn.decomposition.PCA(n_components=3)
decomposition = model.fit_transform(data.transpose(1,2,0).reshape(-1, data.shape[0])).T.reshape(-1, *data.shape[1:])
cluster_labels = np.argmax(decomposition, axis=0)
cluster_label_names = ['Alpha', 'Bravo', 'Charlie']

#### Spectra Annotations

You can add annotations to the spectra plot by providing an annotations dictionary.

In [ ]:
from irviz.viewer import notebook_viewer

annotations = {
    'X1': {
        'range': [3200, 2800],
        'color': 'orange'  # Defaults to gray if no color specified
    },
    'X2': {
        'position': 1000
    },
    'X3': {
        'range': (1500, 2000),
        'position': 1900,
        'color': "#fa00a5"
    },
}

### Calling notebook_viewer

Here, we will be using the `notebook_viewer` function to create our Viewer with the optional arguments we have set up above.


In [ ]:
from irviz.viewer import notebook_viewer

viewer = notebook_viewer(data,
                         annotations=annotations,
                         decomposition=decomposition,
                         bounds=bounds,
                         x_axis_title='X (μm)',
                         y_axis_title='Y (μm)',
                         spectra_axis_title='Wavenumber (cm⁻¹)',
                         intensity_axis_title='Intensity',
                         invert_spectra_axis=True,
                         cluster_labels=cluster_labels,
                         cluster_label_names=cluster_label_names,
                         component_spectra=model.components_,
                         height=1500)

### Viewer Properties Examples



In [ ]:
print(f'current spectrum in the spectra plot:\n\t{viewer.spectrum}',
      f'current spectral value in the spectra plot (i.e. the value at the current x position):\n\t{viewer.spectral_value}',
      f'current x index of the spectra plot:\n\t{viewer.spectral_index}', sep='\n\n')


---

## (TESTING: a second viewer)

In [ ]:
# Create 2nd notebook viewer (after waiting until first has loaded completely)
import time
time.sleep(2)
viewer = notebook_viewer(data, decomposition=decomposition, bounds=bounds, height=1500)